In [39]:
import torch
from torch.autograd import Variable
from torch import Tensor
import numpy as np
import timeit

In [8]:
w = Variable(Tensor(1), requires_grad=True)
w.data[0] = 4.0

b = Variable(Tensor(1), requires_grad=True)
b.data[0] = 4.0

x = Variable(Tensor(1), requires_grad=False)

def forward(x):
    return 3 * w * x + b

In [9]:
for epoch in range(2):
    for val in [2, 2, 2]:
        x.data[0] = val
        out = forward(x)
        out.backward()
        print("out = ", out.data[0])
        print("dw  = ", w.grad.data[0])
        print("db  = ", b.grad.data[0])
        
    w.grad.data.zero_()
    b.grad.data.zero_()
    print("------")

out =  28.0
dw  =  6.0
db  =  1.0
out =  28.0
dw  =  12.0
db  =  2.0
out =  28.0
dw  =  18.0
db  =  3.0
------
out =  28.0
dw  =  6.0
db  =  1.0
out =  28.0
dw  =  12.0
db  =  2.0
out =  28.0
dw  =  18.0
db  =  3.0
------


In [24]:
o = torch.from_numpy(np.array([1.0, 1.0]))
v = torch.from_numpy(np.array([0.0, 0.0]))
torch.dist(v, o, p=2)

1.4142135623730951

In [35]:
t = Tensor([1, 2, 3])
v = Variable(t)
v.volatile

False

In [43]:
### NUMPY ###

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

start_time = timeit.default_timer()
for counter in range(10):

    # Create random input and output data
    x = np.random.randn(N, D_in)
    y = np.random.randn(N, D_out)

    # Randomly initialize weights
    w1 = np.random.randn(D_in, H)
    w2 = np.random.randn(H, D_out)

    learning_rate = 1e-6
    for t in range(500):
        # Forward pass: compute predicted y
        h = x.dot(w1)
        h_relu = np.maximum(h, 0)
        y_pred = h_relu.dot(w2)

        # Compute and print loss
        loss = np.square(y_pred - y).sum()

        # Backprop to compute gradients of w1 and w2 with respect to loss
        grad_y_pred = 2.0 * (y_pred - y)
        grad_w2 = h_relu.T.dot(grad_y_pred)
        grad_h_relu = grad_y_pred.dot(w2.T)
        grad_h = grad_h_relu.copy()
        grad_h[h < 0] = 0
        grad_w1 = x.T.dot(grad_h)

        # Update weights
        w1 -= learning_rate * grad_w1
        w2 -= learning_rate * grad_w2
        
print(timeit.default_timer() - start_time)

6.208205400034785


In [44]:
### PYTORCH ###

dtype = torch.FloatTensor

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

start_time = timeit.default_timer()
for counter in range(10):

    # Create random input and output data
    x = torch.randn(N, D_in).type(dtype)
    y = torch.randn(N, D_out).type(dtype)

    # Randomly initialize weights
    w1 = torch.randn(D_in, H).type(dtype)
    w2 = torch.randn(H, D_out).type(dtype)

    learning_rate = 1e-6
    for t in range(500):
        # Forward pass: compute predicted y
        h = x.mm(w1)
        h_relu = h.clamp(min=0)
        y_pred = h_relu.mm(w2)

        # Compute and print loss
        loss = (y_pred - y).pow(2).sum()

        # Backprop to compute gradients of w1 and w2 with respect to loss
        grad_y_pred = 2.0 * (y_pred - y)
        grad_w2 = h_relu.t().mm(grad_y_pred)
        grad_h_relu = grad_y_pred.mm(w2.t())
        grad_h = grad_h_relu.clone()
        grad_h[h < 0] = 0
        grad_w1 = x.t().mm(grad_h)

        # Update weights using gradient descent
        w1 -= learning_rate * grad_w1
        w2 -= learning_rate * grad_w2
        
print(timeit.default_timer() - start_time)

2.1072698088828474


In [46]:
# PYTORCH AUTOGRAD #

dtype = torch.FloatTensor

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

start_time = timeit.default_timer()
for counter in range(10):

    # Create random Tensors to hold input and outputs, and wrap them in Variables.
    # Setting requires_grad=False indicates that we do not need to compute gradients
    # with respect to these Variables during the backward pass.
    x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
    y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

    # Create random Tensors for weights, and wrap them in Variables.
    # Setting requires_grad=True indicates that we want to compute gradients with
    # respect to these Variables during the backward pass.
    w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
    w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

    learning_rate = 1e-6
    for t in range(500):
        # Forward pass: compute predicted y using operations on Variables; these
        # are exactly the same operations we used to compute the forward pass using
        # Tensors, but we do not need to keep references to intermediate values since
        # we are not implementing the backward pass by hand.
        y_pred = x.mm(w1).clamp(min=0).mm(w2)

        # Compute and print loss using operations on Variables.
        # Now loss is a Variable of shape (1,) and loss.data is a Tensor of shape
        # (1,); loss.data[0] is a scalar value holding the loss.
        loss = (y_pred - y).pow(2).sum()
        
        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Variables with requires_grad=True.
        # After this call w1.grad and w2.grad will be Variables holding the gradient
        # of the loss with respect to w1 and w2 respectively.
        loss.backward()

        # Update weights using gradient descent; w1.data and w2.data are Tensors,
        # w1.grad and w2.grad are Variables and w1.grad.data and w2.grad.data are
        # Tensors.
        w1.data -= learning_rate * w1.grad.data
        w2.data -= learning_rate * w2.grad.data

        # Manually zero the gradients after updating weights
        w1.grad.data.zero_()
        w2.grad.data.zero_()
        
print(timeit.default_timer() - start_time)

5.65750572597608
